# 用于对本月的历史数据按照板块名称进行分组统计，结果保存到result.csv文件

In [69]:
import pandas as pd
from datetime import datetime
# import sys
# sys.path.append('../utils')
# import constants
from functools import lru_cache
OPTION_DICT = {
    "all": (float('-inf'), float('inf')),
    "0-100": (0, 100),
    "100-500": (100, 500),
    "500-1000": (500, 1000),
    "1000-30000": (1000, 30000),
}

RANGE = ["跌停", "跌<-5%",  "-3%<-5%",     "-3<-1%",
         "平盘", "<3%",     "3-5%",   "5%-涨停", "涨停"]


In [70]:
def get_data() -> tuple[pd.DataFrame, list]:
    """
    获得股票历史信息，并计算总市值
    """
    # 显示结果
    df = pd.read_csv(
        f"../data/merge_{datetime.now().strftime('%Y-%m-%d')}.csv", parse_dates=['日期'], index_col=0, dtype={"股票代码": object})
    # dates = df.index.unique().sort_values().to_list()
    # print(type(dates[0]))
    # dates = [x.strftime("%Y-%m-%d") for x in dates]
    # 获得当前结果集的日期列表
    # dates_list = [date.strftime('%Y-%m-%d') for date in dates]
    value = pd.read_csv("../data/总股本_em.csv", index_col="股票代码", dtype={"股票代码": object})
    value_dict = value['总股本'].to_dict()
    df['总股本'] = df['股票代码'].apply(lambda x: value_dict.get(x))
    df['总市值'] = df['总股本']*df['收盘']
    # 把总市值为none的变成0
    # df.fillna(0, inplace=True)
    return df


In [ ]:
print(OPTION_DICT)

In [71]:
df = get_data()
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 170816 entries, 2023-03-01 to 2023-03-24
Data columns (total 15 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   开盘      170816 non-null  float64
 1   收盘      170816 non-null  float64
 2   最高      170816 non-null  float64
 3   最低      170816 non-null  float64
 4   成交量     170816 non-null  int64  
 5   成交额     170816 non-null  float64
 6   振幅      170816 non-null  float64
 7   涨跌幅     170816 non-null  float64
 8   涨跌额     170816 non-null  float64
 9   换手率     170816 non-null  float64
 10  股票代码    170816 non-null  object 
 11  股票名称    170816 non-null  object 
 12  板块名称    170816 non-null  object 
 13  总股本     170816 non-null  float64
 14  总市值     170816 non-null  float64
dtypes: float64(11), int64(1), object(3)
memory usage: 20.9+ MB


In [72]:
df.isnull().sum()

开盘      0
收盘      0
最高      0
最低      0
成交量     0
成交额     0
振幅      0
涨跌幅     0
涨跌额     0
换手率     0
股票代码    0
股票名称    0
板块名称    0
总股本     0
总市值     0
dtype: int64

In [ ]:
df.columns

In [ ]:
df.head()

In [73]:
counts = df.groupby('板块名称')['股票名称'].count()
counts

板块名称
IT服务    2286
LED      720
专业工程     612
专业服务     496
专业连锁     144
        ... 
饮料制造     846
饰品       288
饲料       320
高速公路     360
黑色家电     198
Name: 股票名称, Length: 288, dtype: int64

In [74]:
# 计算每个板块下有多少股票
counts = df.groupby('板块名称')['股票名称'].nunique()
counts.columns=["板块名称", "股票个数"]
# 打印结果
print(counts)


板块名称
IT服务    127
LED      40
专业工程     34
专业服务     28
专业连锁      8
       ... 
饮料制造     47
饰品       16
饲料       18
高速公路     20
黑色家电     11
Name: 股票名称, Length: 288, dtype: int64


In [75]:
start_value, end_value = 0,100
cur_df = df.copy()
cur_df = cur_df[(cur_df['总市值'] >= (start_value)*100_000_000)
                    & (cur_df['总市值'] <= (end_value)*100_000_000)]

# data = df.groupby("板块名称")["股票代码"].count()
# 计算每个板块下有多少股票
counts = cur_df.groupby('板块名称')['股票名称'].nunique()
# counts.reset_index(inplace=True)
counts.name = "股票个数"
counts


板块名称
IT服务    92
LED     36
专业工程    24
专业服务    23
专业连锁     4
        ..
饮料制造    22
饰品      12
饲料      11
高速公路    12
黑色家电     5
Name: 股票个数, Length: 277, dtype: int64

In [79]:
da=cur_df[cur_df["板块名称"] == "农业服务"]
da.loc['2023-03-01']

,开盘,收盘,最高,最低,成交量,成交额,振幅,涨跌幅,涨跌额,换手率,股票代码,股票名称,板块名称,总股本,总市值
日期,,,,,,,,,,,,,,,
2023-03-01,4.43,4.45,4.48,4.40,99091,4.394215e+07,1.81,0.68,0.03,0.87,002141,贤丰控股,农业服务,1.134657e+09,5.049222e+09
2023-03-01,4.98,4.96,4.99,4.94,132872,6.598454e+07,1.00,-0.40,-0.02,1.90,300021,大禹节水,农业服务,8.622998e+08,4.277007e+09
2023-03-01,8.37,8.36,8.38,8.32,24992,2.087038e+07,0.72,0.00,0.00,0.61,688098,申联生物,农业服务,4.106440e+08,3.432984e+09
2023-03-01,21.79,21.84,22.08,21.66,15582,3.401650e+07,1.92,-0.27,-0.06,1.88,300871,回盛生物,农业服务,1.660545e+08,3.626631e+09
2023-03-01,11.58,11.54,11.60,11.44,29497,3.396033e+07,1.39,0.00,0.00,0.46,603718,海利生物,农业服务,6.440000e+08,7.431760e+09
2023-03-01,8.81,8.90,8.90,8.78,69040,6.110288e+07,1.36,0.91,0.08,0.74,002556,辉隆股份,农业服务,9.539930e+08,8.490538e+09
2023-03-01,3.42,3.39,3.42,3.35,1772,5.953977e+05,2.06,-0.29,-0.01,0.20,838275,驱动力,农业服务,1.602776e+08,5.433411e+08
2023-03-01,8.53,8.45,8.53,8.26,354,2.985342e+05,3.20,0.24,0.02,0.04,839729,永顺生物,农业服务,2.733500e+08,2.309808e+09
2023-03-01,3.44,3.46,3.46,3.44,1434,4.953424e+05,0.58,0.00,0.00,0.08,830964,润农节水,农业服务,2.612080e+08,9.037797e+08


In [76]:
counts.loc['农业服务']


13

## 正式运行

In [ ]:
for key,value in OPTION_DICT.items():
    start_value, end_value = value
    cur_df = df.copy()
    if key!="all":
        cur_df = cur_df[(cur_df['总市值'] >= (start_value)*100_000_000)
                        & (cur_df['总市值'] <= (end_value)*100_000_000)]
    
    # data = df.groupby("板块名称")["股票代码"].count()
    # 计算每个板块下有多少股票
    counts = cur_df.groupby('板块名称')['股票名称'].nunique()
    # counts.reset_index(inplace=True)
    counts.name = "股票个数"
    counts.to_csv(
        f"../data/股票代码个数_{key}.csv", index=True)


In [ ]:
data = pd.read_csv(
    f"../data/股票代码个数_all.csv", index_col=0, )
# data.reset_index(inplace=True)
# data = df.groupby("板块名称")["股票代码"].count()
# data_dict = data.to_dict()
data["股票个数"].to_dict()
